In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import sklearn
import warnings
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from pylab import rcParams

In [62]:
data=pd.read_csv('Auto.csv', na_values='?')
data=data.dropna()
data.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [84]:

data['horsepower'] = data['horsepower'].convert_objects(convert_numeric = True)
#create dummy variable below, based on median value of mpg
data['mpg_high'] = data['mpg'].apply(lambda x: 1 if x >= data['mpg'].median() else 0).astype('category')

data['const'] = 1
xvars = data[['const','cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'year', 'origin']].values
yvals = data['mpg_high'].values

/Users/sumervaid/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


# Question 1) Part a):

In [86]:

k=4
kfold=KFold(n_splits=k, random_state=9, shuffle=True)
kfold.get_n_splits(xvars)
error0=np.zeros(k)
error1=np.zeros(k)
errormse=np.zeros(k)

k_ind=int(0)


for train_index, test_index in kfold.split(xvars):
    print("When k=", k_ind)
    X_train, X_test=xvars[train_index], xvars[test_index]
    y_train, y_test=yvals[train_index], yvals[test_index]
    Reg=LogisticRegression(fit_intercept=True)
    Reg.fit(X_train, y_train)
    ypred=Reg.predict(X_test)
    error=y_test!=ypred
    error_all_class=error.mean()
    error_0=((ypred==0)*error).sum()/(ypred==0).sum()
    error_1=((ypred==1)*error).sum()/(ypred==1).sum()
    error0[k_ind]=error_0
    error1[k_ind]=error_1
    errormse=error_all_class
    print("The error rate (mpghigh= 0) is", error_0)
    print("The error rate (mpghigh= 1)  is" ,error_1)
    print("The MSE of the model is", error_all_class)
    print(classification_report(y_test, ypred, digits=3))

    k_ind += 1
    print("\nk-fold results:")
    print("The avg. error rate (mpghigh=0) is", error0.mean(), "and the stdev is", error0.std())
    print("The avg. error rate (mpghigh= 1) is", error1.mean(), "and the stdev is", error1.std())
    print("The avg. MSE is", errormse.mean(), "and the stdev is", errormse.std())
    


When k= 0
The error rate (mpghigh= 0) is 0.08
The error rate (mpghigh= 1)  is 0.0833333333333
The MSE of the model is 0.0816326530612
             precision    recall  f1-score   support

          0      0.920     0.920     0.920        50
          1      0.917     0.917     0.917        48

avg / total      0.918     0.918     0.918        98


k-fold results:
The avg. error rate (mpghigh=0) is 0.02 and the stdev is 0.0346410161514
The avg. error rate (mpghigh= 1) is 0.0208333333333 and the stdev is 0.0360843918244
The avg. MSE is 0.0816326530612 and the stdev is 0.0
When k= 1
The error rate (mpghigh= 0) is 0.133333333333
The error rate (mpghigh= 1)  is 0.11320754717
The MSE of the model is 0.122448979592
             precision    recall  f1-score   support

          0      0.867     0.867     0.867        45
          1      0.887     0.887     0.887        53

avg / total      0.878     0.878     0.878        98


k-fold results:
The avg. error rate (mpghigh=0) is 0.0533333333333

# Question 1) Part b)

In [87]:
rf=RandomForestClassifier(n_estimators=20, max_features=2,bootstrap=True, oob_score=True, random_state=25)
rf.fit(xvars, yvals)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [88]:
decision=rf.oob_decision_function_[:,1]
binary=np.vectorize(lambda xvars: int(xvars>=.5))
ypred=binary(decision)
mse_all=mean_squared_error(yvals, ypred)
print("The MSE of the model is", mse_all)
treedf=pd.DataFrame({'y':yvals, 'ypred': ypred})
treedf['MSE']=treedf.apply(lambda x:(x['y']-x['ypred'])**2, axis=1)
print("MSE for mpg_high=1 is", treedf.groupby(yvals)["MSE"].mean()[1])
print("MSE for mpg_high=0 is", treedf.groupby(yvals)["MSE"].mean()[0])

The MSE of the model is 0.0790816326531
MSE for mpg_high=1 is 0.0714285714286
MSE for mpg_high=0 is 0.0867346938776


# Question 1 Part c)

In [89]:
k=4
kfold=KFold(n_splits=k, random_state=15, shuffle=True)
kfold.get_n_splits(xvars)
svmerr0=np.zeros(k)
svmerr1=np.zeros(k)
svm_mse=np.zeros(k)
k_ind=int(0)

for train_index, test_index in kfold.split(xvars):
    print('When k index=', k_ind)
    x_train, x_test=xvars[train_index], xvars[test_index]
    y_train, y_test=yvals[train_index], yvals[test_index]
    svc=svm.SVC(kernel='rbf', gamma=0.2, C=1)
    svc.fit(x_train, y_train)
    y_pred=svc.predict(x_test)
    error=y_test!=y_pred
    errorall=error.mean()
    error_0=((y_pred==0)*error).sum()/(y_pred==0).sum()
    error_1=((y_pred==1)*error).sum()/(y_pred==1).sum()
    svmerr0[k_ind]=error_0
    svmerr1[k_ind]=error_1
    svm_mse[k_ind]=((y_test-y_pred)**2).mean()
    print("The MSE for the test set", k_ind, "is",svmgeneral_mse[k_ind])
    k_ind+=1
    
msekf=svmgeneral_mse.mean()
kfstd=svmgeneral_mse.std()
print("The MSE k-fold estimate is", msekf)
print("The MSE stdev is", kfstd)

When k index= 0
The MSE for the test set 0 is 0.0
When k index= 1
The MSE for the test set 1 is 0.0
When k index= 2
The MSE for the test set 2 is 0.0
When k index= 3
The MSE for the test set 3 is 0.0
The MSE k-fold estimate is 0.0
The MSE stdev is 0.0


# Question 1d)


I am unable to obtain the correct SVC error values for some reason. I would assume that under usual cases, either Random Forest or logistic regression would provide a better fit than SVC. 